In [2]:
%matplotlib inline
%reset
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import scipy

import SchemDraw as schem # to draw schematics (install using 'pip install SchemDraw')
import SchemDraw.elements as e

import skrf as rf # scikit-rf (install using 'conda -c scikit-rf scikit-rf')

import sympy as s # sympy (install using 'conda -c sympy sympy')
from sympy import I # imaginary symbol

rf.stylely() # enable scikit-rf plot styles
s.init_printing(use_latex='mathjax') # print sympy equations in MathJAX (for Jupyter)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [18]:
# Single tone harmonic distortion
A, w1, t = s.symbols('A \omega_1 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
display(s.expand(s.fu(So, measure=lambda x: -x.count_ops())))

A⋅cos(\omega₁⋅t)

   3                      3                        2                        2 
3⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   A ⋅a₂⋅cos(2⋅\omega₁⋅t)   A ⋅
────────────────────── + ────────────────────── + ────────────────────── + ───
          4                        4                        2                2

                        
a₂                      
── + A⋅a₁⋅cos(\omega₁⋅t)
                        

In [20]:
# Two tone harmonic distortion
A, B, w1, w2, t = s.symbols('A B \omega_1 \omega_2 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t) + B * s.cos(w2*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
So = s.expand(s.fu(So, measure=lambda x: -x.count_ops()))
display(So)
print(s.latex(So))

A⋅cos(\omega₁⋅t) + B⋅cos(\omega₂⋅t)

   3                      3                          2                        
3⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   3⋅A ⋅B⋅a₃⋅cos(\omega₂⋅t)   3
────────────────────── + ────────────────────── + ──────────────────────── + ─
          4                        4                         2                

  2                                        2                                  
⋅A ⋅B⋅a₃⋅cos(2⋅\omega₁⋅t - \omega₂⋅t)   3⋅A ⋅B⋅a₃⋅cos(2⋅\omega₁⋅t + \omega₂⋅t)
───────────────────────────────────── + ──────────────────────────────────────
                 4                                        4                   

    2                        2           2                          2         
   A ⋅a₂⋅cos(2⋅\omega₁⋅t)   A ⋅a₂   3⋅A⋅B ⋅a₃⋅cos(\omega₁⋅t)   3⋅A⋅B ⋅a₃⋅cos(\
 + ────────────────────── + ───── + ──────────────────────── + ───────────────
             2                2                2                              

                               2                 

\frac{3 a_{3}}{4} A^{3} \cos{\left (\omega_1 t \right )} + \frac{A^{3} a_{3}}{4} \cos{\left (3 \omega_1 t \right )} + \frac{3 a_{3}}{2} A^{2} B \cos{\left (\omega_2 t \right )} + \frac{3 a_{3}}{4} A^{2} B \cos{\left (2 \omega_1 t - \omega_2 t \right )} + \frac{3 a_{3}}{4} A^{2} B \cos{\left (2 \omega_1 t + \omega_2 t \right )} + \frac{A^{2} a_{2}}{2} \cos{\left (2 \omega_1 t \right )} + \frac{A^{2} a_{2}}{2} + \frac{3 a_{3}}{2} A B^{2} \cos{\left (\omega_1 t \right )} + \frac{3 a_{3}}{4} A B^{2} \cos{\left (\omega_1 t - 2 \omega_2 t \right )} + \frac{3 a_{3}}{4} A B^{2} \cos{\left (\omega_1 t + 2 \omega_2 t \right )} + A B a_{2} \cos{\left (\omega_1 t - \omega_2 t \right )} + A B a_{2} \cos{\left (\omega_1 t + \omega_2 t \right )} + A a_{1} \cos{\left (\omega_1 t \right )} + \frac{3 a_{3}}{4} B^{3} \cos{\left (\omega_2 t \right )} + \frac{B^{3} a_{3}}{4} \cos{\left (3 \omega_2 t \right )} + \frac{B^{2} a_{2}}{2} \cos{\left (2 \omega_2 t \right )} + \frac{B^{2} a_{2}}{2} + B a_{1} \cos{